In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 3 : formalization: get started with BigQuery ML pipeline components

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_bqml_pipeline_components.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_bqml_pipeline_components.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 3 : formalization: get started with BigQuery ML pipeline components.

### Dataset

The dataset used for this tutorial is the Penguins dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset predicts the species.

### Objective

In this tutorial, you learn how to use prebuilt `Google Cloud Pipeline Components` for `BigQuery ML`.

This tutorial uses the following Google Cloud ML services:

- `BigQuery ML`
- `Google Cloud Pipeline Components`
- `Vertex AI Dataset, Model and Endpoint` resources
- `Vertex AI Prediction`

The steps performed include:

- Construct a pipeline for:
    - Training BigQuery ML model.
    - Evaluating the BigQuery ML model.
    - Exporting the BigQuery ML model.
    - Importing the BigQuery ML model to a Vertex AI model.
    - Deploy the Vertex AI model.
- Execute a Vertex AI pipeline.
- Make a prediction with the deployed Vertex AI model.

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installations

Install *one time* the packages for executing the MLOps notebooks.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [1]:
! pip3 install -U tensorflow $USER_FLAG
! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG
! pip3 install --upgrade google-cloud-bigquery $USER_FLAG
! pip3 install --upgrade kfp $USER_FLAG


     |████████████████████████████████| 497.6 MB 7.6 kB/s  eta 0:00:015     |███▍                            | 53.0 MB 61.0 MB/s eta 0:00:08MB 61.0 MB/s eta 0:00:07     |█████████▍                      | 146.6 MB 68.3 MB/s eta 0:00:06█▏                    | 174.4 MB 57.4 MB/s eta 0:00:06     |██████████████████████████▊     | 416.0 MB 46.5 MB/s eta 0:00:02
     |████████████████████████████████| 4.5 MB 45.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 67.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 52.0 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 42.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 54.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.3 MB/s  eta 0:00:01
     |███████████████

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: vertex-ai-dev


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



To take a quick anonymous survey, run:
  $ gcloud survey



#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "[your-region]"  # @param {type:"string"}
if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [6]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [7]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [8]:
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://vertex-ai-devaip-20220405062906/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [9]:
! gsutil ls -al $BUCKET_URI

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

*Note:* The code for automatically finding your service account works on a user-managed Workbench AI noteboook. If you are using a fully-managed notebook, you will need to manually enter your service account.

In [10]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [11]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 931647533046-compute@developer.gserviceaccount.com


#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [12]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [13]:
import google.cloud.aiplatform as aip

In [14]:
import json

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

#### Import BigQuery

Import the BigQuery package into your Python environment.

In [15]:
from google.cloud import bigquery

#### Import TensorFlow

Import the TensorFlow package into your Python environment.

In [16]:
import tensorflow as tf

2022-04-05 06:29:45.902172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-05 06:29:45.902209: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [17]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### Create BigQuery client

Create the BigQuery client.

In [18]:
bqclient = bigquery.Client()

#### Set hardware accelerators

You can set hardware accelerators for prediction.

Set the variable `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more [here](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) hardware accelerator support for your region

In [19]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.

- Set the variable `TF` to the TensorFlow version of the container image. For example, `2-1` would be version 2.1, and `1-15` would be version 1.15. The following list shows some of the pre-built images available:


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [20]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

Deployment: us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-5:latest AcceleratorType.NVIDIA_TESLA_K80


#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [21]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Construct pipeline for BigQuery ML training, exporting and deployment

Next, construct the pipeline with the following tasks:

- Create a BigQuery schema for the dataset.
- Train a BigQuery ML model.
- Evaluate the BigQuery ML model.
- Make a batch prediction with the BigQuery ML model. *Note:* The same data (table) is used for both training and prediction.
- Export the BigQuery ML model.
- Import the exported model artifacts to a Vertex AI Model resource.
- Deploy the Vertex AI Model resource to a Vertex AI Endpoint resource.

In [22]:
PIPELINE_ROOT = f"{BUCKET_URI}/bq_query"


@dsl.pipeline(name="bq-hello-world", pipeline_root=PIPELINE_ROOT)
def pipeline(
    bq_table: str,
    label: str,
    dataset: str,
    model: str,
    artifact_uri: str,
    num_trials: int,
    deploy_image: str,
    machine_type: str,
    min_replica_count: int,
    max_replica_count: int,
    display_name: str,
    accelerator_type: str = "",
    accelerator_count: int = 0,
    project: str = PROJECT_ID,
    location: str = "US",
    region: str = "us-central1",
):
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.bigquery import (
        BigqueryCreateModelJobOp, BigqueryEvaluateModelJobOp,
        BigqueryExportModelJobOp, BigqueryPredictModelJobOp,
        BigqueryQueryJobOp)
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.v2.components import importer_node

    bq_dataset = BigqueryQueryJobOp(
        project=project, location="US", query=f"CREATE SCHEMA {dataset}"
    )

    bq_model = BigqueryCreateModelJobOp(
        project=project,
        location=location,
        query=f"CREATE OR REPLACE MODEL {dataset}.{model} OPTIONS (model_type='dnn_classifier', labels=['{label}'], num_trials={num_trials}) AS SELECT * FROM `{bq_table}` WHERE body_mass_g IS NOT NULL AND sex IS NOT NULL",
    ).after(bq_dataset)

    bq_eval = BigqueryEvaluateModelJobOp(
        project=PROJECT_ID, location="US", model=bq_model.outputs["model"]
    ).after(bq_model)

    bq_predict = BigqueryPredictModelJobOp(
        project=project,
        location=location,
        model=bq_model.outputs["model"],
        table_name=f"`{bq_table}`",
        # query_statement=f"SELECT * EXCEPT ({label}) FROM {bq_table} WHERE body_mass_g IS NOT NULL AND sex IS NOT NULL"
        job_configuration_query={
            "destinationTable": {
                "projectId": PROJECT_ID,
                "datasetId": "bqml_tutorial",
                "tableId": "results_1",
            }
        },
    ).after(bq_model)

    bq_export = BigqueryExportModelJobOp(
        project=project,
        location=location,
        model=bq_model.outputs["model"],
        model_destination_path=artifact_uri,
    ).after(bq_model)

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=artifact_uri,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": DEPLOY_IMAGE,
            },
        },
    ).after(bq_export)

    model_upload = ModelUploadOp(
        project=project,
        display_name=display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    ).after(import_unmanaged_model_task)

    endpoint = EndpointCreateOp(
        project=project,
        location=region,
        display_name=display_name,
    ).after(model_upload)

    deploy_model = ModelDeployOp(
        model=model_upload.outputs["model"],
        endpoint=endpoint.outputs["endpoint"],
        dedicated_resources_min_replica_count=min_replica_count,
        dedicated_resources_max_replica_count=max_replica_count,
        dedicated_resources_machine_type=machine_type,
        dedicated_resources_accelerator_type=accelerator_type,
        dedicated_resources_accelerator_count=accelerator_count,
        traffic_split={"0": 100},
    )

### Compile and execute the BigQuery ML train, export and deploy pipeline

Next, you compile the pipeline and then execute it. The pipeline takes the following parameters, which are passed as the dictionary `parameter_values`:

- `bq_table`: The BigQuery dataset to train on.
- `label`: The corresponding label for the BigQuery dataset.
- `dataset`: The BigQuery dataset component name.
- `model`: The BigQuery model component name.
- `artifact_uri`: The Cloud Storage location to export the BigQuery model artifacts.
- `num_trials`: If greater than one, will perform hyperparameter tuning for the specified number of trials using the Vertex AI Vizier service.
- `deploy_image`: The container image for serving predictions.
- `machine_type`: The VM for serving predictions.
- `min_replica_count`/`max_replica_count`: The number of virtual machines for auto-scaling predictions.
- `accelerator_type`: The type of HW accelerators -- if any.
- `accelerator_count`: The number of HW accelerators -- if any.
- `display_name`: Display name for Vertex AI Model and Endpoint resources.
- `project`: The project ID
- `location`: The location

In [23]:
#If DEPLOY_GPU is None, keeping gpu as no accelerator and accelerator_count as 0
accelerator_count=0
if DEPLOY_GPU:
    gpu = DEPLOY_GPU.name
    accelerator_count=1 
else:
    gpu = "ACCELERATOR_TYPE_UNSPECIFIED" #Unspecified accelerator type, which means no accelerator.
    accelerator_count=0  
    
print("gpu=",gpu)
print("accelerator_count=",accelerator_count)

MODEL_DIR = BUCKET_URI + "/bqmodel"

compiler.Compiler().compile(pipeline_func=pipeline, package_path="bqml.json")

pipeline = aip.PipelineJob(
    display_name="bqml",
    template_path="bqml.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "bq_table": BQ_TABLE,
        "label": "species",
        "dataset": "bqml_tutorial",
        "model": "penguins_model",
        "artifact_uri": MODEL_DIR,
        "num_trials": 2,
        "deploy_image": DEPLOY_IMAGE,
        "display_name": "penguins",
        "machine_type": "n1-standard-4",
        "min_replica_count": 1,
        "max_replica_count": 1,
        "accelerator_type": gpu,
        "accelerator_count": accelerator_count,
        "project": PROJECT_ID,
        "location": "US",
    },
    enable_caching=False,
)

pipeline.run()

! rm -rf bqml.json

gpu= NVIDIA_TESLA_K80
acc_count= 1


/usr/local/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1278: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/931647533046/locations/us-central1/pipelineJobs/bq-hello-world-20220405063042
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/931647533046/locations/us-central1/pipelineJobs/bq-hello-world-20220405063042')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bq-hello-world-20220405063042?project=931647533046
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/931647533046/locations/us-central1/pipelineJobs/bq-hello-world-20220405063042 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/931647533046/locations/us-central1/pipelineJobs/bq-hello-

### View the BigQuery ML pipeline results

In [24]:
PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


print("bigquery-query-job")
artifacts = print_pipeline_output(pipeline, "bigquery-query-job")
print("\n\n")
print("bigquery-create-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-create-model-job")
print("\n\n")
print("bigquery-evaluate-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-evaluate-model-job")
print("\n\n")
print("bigquery-predict-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-predict-model-job")
print("\n\n")
print("bigquery-export-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-export-model-job")
print("\n\n")
print("model-upload")
artifacts = print_pipeline_output(pipeline, "model-upload")
output = !gsutil cat $artifacts
output = json.loads(output[0])
model_id = output["artifacts"]["model"]["artifacts"][0]["metadata"]["resourceName"]
print("\n\n")
print("endpoint-create")
artifacts = print_pipeline_output(pipeline, "endpoint-create")
print("\n\n")
output = !gsutil cat $artifacts
output = json.loads(output[0])
endpoint_id = output["artifacts"]["endpoint"]["artifacts"][0]["metadata"][
    "resourceName"
]
print("\n")
print(endpoint_id)
print("\n")
print("model-deploy")
artifacts = print_pipeline_output(pipeline, "model-deploy")
print("\n\n")

931647533046
bigquery-query-job
bq-hello-world-20220405063042
{
  "resources": [
    {
      "resourceType": "BigQueryJob",
      "resourceUri": "https://www.googleapis.com/bigquery/v2/projects/vertex-ai-dev/jobs/job_103pMpXKB4IsXwhgwcpiOAlzaO9M?location=US"
    }
  ]
}


bigquery-create-model-job
bq-hello-world-20220405063042
{"artifacts": {"model": {"artifacts": [{"metadata": {"projectId": "vertex-ai-dev", "datasetId": "bqml_tutorial", "modelId": "penguins_model"}, "name": "projects/931647533046/locations/us-central1/metadataStores/default/artifacts/14029859898460212712", "type": {"schemaTitle": "google.BQMLModel"}, "uri": "https://www.googleapis.com/bigquery/v2/projects/vertex-ai-dev/datasets/bqml_tutorial/models/penguins_model"}]}}}


bigquery-evaluate-model-job
bq-hello-world-20220405063042
{"artifacts": {"evaluation_metrics": {"artifacts": [{"metadata": {"schema": {"fields": [{"name": "trial_id", "type": "INTEGER", "mode": "NULLABLE"}, {"name": "precision", "type": "FLOAT", "mode

### Delete a pipeline job

After a pipeline job is completed, you can delete the pipeline job with the method `delete()`.  Prior to completion, a pipeline job can be canceled with the method `cancel()`.

In [25]:
pipeline.delete()

INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/931647533046/locations/us-central1/pipelineJobs/bq-hello-world-20220405063042
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/931647533046/locations/us-central1/operations/3506908348429631488
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/931647533046/locations/us-central1/pipelineJobs/bq-hello-world-20220405063042


#### Load an endpoint

The 'Endpoint' initializer will load an endpoint from an endpoint identifier.

In [26]:
endpoint = aip.Endpoint(endpoint_id)

#### Make prediction instances

Next, you prepare a prediction request using a synthetic example. The format for making a prediction request to an export BigQuery ML model is dependent on the exported model format. In the case where `model_type=DNN_CLASSIFIER`, the exported model format is a TensorFlow estimator format. For this format, you use the `raw_predict()`, with the following request format:

    http_body -> {
        'signature_name' : serving_signature,
        'instances': [ {instance_1}, {instance_2}, ... ]
    }

    instance -> { 'feature_1': value_1, 'feature_2': value_2, ... }

    serving_signature -> "predict"

Below is a partial list of mapping BigQuery ML model types to their corresponding exported model format:

'LINEAR_REG'<br/>
'LOGISTIC_REG' -> TensorFlow SavedFormat

'AUTOML_CLASSIFIER'<br/>
'AUTOML_REGRESSOR' -> TensorFlow SavedFormat

'BOOSTED_TREE_CLASSIFIER'<br/>
'BOOSTED_TREE_REGRESSOR' -> XGBoost format

'DNN_CLASSIFIER'<br/>
'DNN_REGRESSOR'<br/>
'DNN_LINEAR_COMBINED_CLASSIFIER'<br/>
'DNN_LINEAR_COMBINED_REGRESSOR' -> TensorFlow Estimator

In [27]:
import json

from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

DATA = {
    "signature_name": "predict",
    "instances": [
        {
            "island": "DREAM",
            "culmen_length_mm": 36.6,
            "culmen_depth_mm": 18.4,
            "flipper_length_mm": 184.0,
            "body_mass_g": 3475.0,
            "sex": "FEMALE",
        }
    ],
}

http_body = httpbody_pb2.HttpBody(
    data=json.dumps(DATA).encode("utf-8"),
    content_type="application/json",
)

req = aiplatform_v1.RawPredictRequest(http_body=http_body, endpoint=endpoint_id)

### Make a prediction

Finally, you make the prediction request using Vertex AI Prediction service.

In [28]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}

pred_client = aip.gapic.PredictionServiceClient(client_options=client_options)

response = pred_client.raw_predict(req)
print(response)

content_type: "application/json"
data: "{\n    \"predictions\": [\n        {\n            \"class_ids\": [2],\n            \"classes\": [\"Adelie Penguin (Pygoscelis adeliae)\"],\n            \"all_class_ids\": [0, 1, 2],\n            \"all_classes\": [\"Gentoo penguin (Pygoscelis papua)\", \"Chinstrap penguin (Pygoscelis antarctica)\", \"Adelie Penguin (Pygoscelis adeliae)\"],\n            \"probabilities\": [0.248956278, 0.351945698, 0.399097979],\n            \"logits\": [-0.0984064, 0.247793272, 0.373523355]\n        }\n    ]\n}"
extensions {
  type_url: "type.googleapis.com/google.rpc.context.HttpHeaderContext"
  value: "\030\310\001"
}



#### Delete the BigQuery model and dataset

Next, delete the BigQuery model and dataset.

In [29]:
try:
    job = bqclient.delete_model("bqml_tutorial.penguins_model")
except:
    pass
job = bqclient.delete_dataset("bqml_tutorial", delete_contents=True)

#### Delete the Vertex AI Model and Endpoint resources

Next, undelpoy and delete the Vertex AI Model and Endpoint resources.

In [30]:
endpoint.undeploy_all()
endpoint.delete()

model = aip.Model(model_id)
model.delete()

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/931647533046/locations/us-central1/endpoints/4441226531750019072
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/4441226531750019072/operations/6931895865044893696
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/931647533046/locations/us-central1/endpoints/4441226531750019072
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/931647533046/locations/us-central1/endpoints/4441226531750019072
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/931647533046/locations/us-central1/operations/6679694285912145920
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/931647533046/locations/us-central1/endpoints/4441226531750019072
INFO:google.cloud.aiplatform.base:Deleting Model : projects/931647533046/locations/us-central1/models/8958666811490959360


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [31]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

E0405 07:13:33.358445421     181 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Removing gs://vertex-ai-devaip-20220405062906/bq_query/#1649140312924297...
Removing gs://vertex-ai-devaip-20220405062906/bq_query/931647533046/#1649140313136542...
Removing gs://vertex-ai-devaip-20220405062906/bq_query/931647533046/bq-hello-world-20220405063042/#1649140313322952...
Removing gs://vertex-ai-devaip-20220405062906/bq_query/931647533046/bq-hello-world-20220405063042/bigquery-create-model-job_5529392299038998528/#1649140428291709...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://vertex-ai-devaip-20220405062906/bq_query/931647533046/bq-hello-world-20220405063042/bigquery-create-model-job_5529392299038998528/executor_output.json#1649141495321744...
Removing gs://vertex-ai-devaip-20